# Chambolle and Pock primal-dual algorithm for convex problems

The purpose of this notebook is to implement the Algorithm of Chambolle and Pock and apply it for an inpaiting problem.

In [ ]:
import numpy as np
from numpy.random import randn
import matplotlib.image as mpimg 
import matplotlib.pyplot as plt
import numpy.linalg as npl 
from numba import vectorize, jit, cuda 
import numpy as np 
from timeit import default_timer as timer 
import time

from __future__ import division
from numpy import random
import scipy as scp
import pylab as pyl
import pywt
from PIL import Image
import requests
from io import BytesIO
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%load_ext autoreload
%autoreload 2

The general problem considered in the work of Chambolle and Pock is the generic saddle-point problem given by

$$\text{min}_{x\in X} \text{max}_{y\in Y} \langle Kx, Y \rangle_1 + G(x) - F^*(y) $$

where $G : X \longrightarrow [0,+\infty)$ and $F^* : Y \longrightarrow [0,+\infty)$ are proper, convex, lower_semicontinous (l.s.c) functions and $F^*$ is the convex conjugate of a onvex l.s.c. function $F$.

This saddle-point problem is a primal-dual formulation of the nonlinear primal problem

$$\text{min}_{x\in X} F(Kx)+ G(x) $$

or of the corresponding dual problem

$$\text{max} -(G^*(-K^*y) + F^*(y))$$

The initialization of the algorithm consists in choosing $\sigma, \tau >0$, $\theta\in [0,1]$, $(x^0,y^0)\in X\times Y$ and setting $\bar{x}^0=x^0$, and the iterations are given by

\begin{equation}
\left\{
\begin{matrix}
y^{n+1}=&(I+\sigma\partial F^*)^{-1}(y^n+\sigma K \bar{x}^n)\\
x^{n+1}=&(I+\tau\partial G)^{-1}(x^n-\tau K^* y^{n+1})\\
\bar{x}^{n+1}=&x^{n+1} + \theta (x^{n+1}-x^n)
\end{matrix}
\right.
\end{equation}

To ensure the convergence of this algorithm, $\sigma$ $\tau$ have to be chosen such that $\tau\sigma L^2<1$ where $L=\|K\|$.

# Inpainting using total variation

In this section we treat an inpainting problem which corresponds to filling holes in an image.

Considering measurements $y=\Phi u_0$ where $\Phi$ is a masking operator, the problem can be written as 
$$\text{min}_u \| \Phi u \|_1 + \frac{\lambda}{2}\|u-y\|_2^2. $$
where $\Phi$ denotes a linear transformation.

First we load the image to be inpainted.

In [ ]:
url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Lenna.jpg'        
response = requests.get(url)
u0=np.array(Image.open(BytesIO(response.content))).astype(float)
u0 = u0/255

plt.imshow(u0, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
plt.show()
n = np.shape(u0)[0]

Now to produce an observation, we construct a mask $\Omega$ made of random pixel locations where the perameter $\rho$ is used to choose the amout of removed pixels. The damaging operator put to zeros the pixel locations $x$ for which $\Omega(x)=1$. The damaged observations reads $y = \Phi u_0$.

In [ ]:
rho = .7
Omega = np.zeros([n, n])
sel = random.permutation(n**2)
np.ravel(Omega)[sel[np.arange(int(rho*n**2))]] = 1
Phi = lambda f, Omega: f*(1-Omega)
inp_img = Phi(u0, Omega)
plt.imshow(inp_img, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
plt.show()

To solve the problem using the algorithm of Chamboll et Pock, we consider the following modification. We denote by $D=\{(i,j), 1 \leq i \leq M, 1 \leq j \leq N \}$ the set of indices of the image domain and by $I\subset D$ the set of indices of the inpainting domain.  The inpainting probblem is now given by 
$$\text{min}_{u \in X} \| \Phi u \|_1 + \frac{\lambda}{2}\sum_{i,j\in D\setminus I}(u_{i,j}-y_{i,j})^2. $$
The saddle-point formulation of the problem is given by 
$$\text{min}_{u\in X} \text{max}_{c\in W} \langle \Phi u , c \rangle_1 + \frac{\lambda}{2}\sum_{i,j\in D\setminus I}(u_{i,j}-y_{i,j})^2 - \delta_C(c) $$
where $C$ is the convex set defined by 
$$C = \{c \in W : \|c\|_{\infty} \leq 1 \}.$$
To keep the same notations as the initial algorithm, we set $G(u) = \frac{\lambda}{2}\sum_{i,j\in D\setminus I}(u_{i,j}-y_{i,j})^2$ and $F^*(c)= \delta_C(c)$.

Thus the resolvent operators are given by 
$$c = (I+\sigma\partial F^*)^{-1}(\tilde{c}) \Leftrightarrow c_k = \frac{\tilde{c}_k}{\text{max}(1,|\tilde{c}_k|)}$$
and 
$$u = (I+\tau\partial G)^{-1}(\tilde{u}) \Leftrightarrow u_{i,j} = \tilde{u}_{i,j} \text{ if } (i,j) \in I, \frac{\tilde{u}_{i,j}+\tau\lambda g_{i,j}}{1+\tau\lambda} \text{else}.$$

The total variation regularisation corresponds to considering the linear transformation $\Phi$ to be the gradient operator.

In [ ]:
def GradientHor(x):
    y=x-np.roll(x,1,axis=1)
    y[:,0]=0
    return y
def GradientVer(x):
    y=x-np.roll(x,1,axis=0)
    y[0,:]=0
    return y
def DivHor(x):
    N=len(x[0])
    y=x-np.roll(x,-1,axis=1)
    y[:,0]=-x[:,1]
    y[:,N-1]=x[:,N-1]
    return y
def DivVer(x):
    N=len(x)
    y=x-np.roll(x,-1,axis=0)
    y[0,:]=-x[1,:]
    y[N-1,:]=x[N-1,:]
    return y
def Gradient(x):
    y=[]
    y.append(GradientHor(x))
    y.append(GradientVer(x))
    return np.array(y)
def Div(y):
    x=DivHor(y[0]) + DivVer(y[1])
    return np.array(x)
def ProjGradBouleInf(g,l):
    gh=g[0]
    gv=g[1]
    temp=g
    p0=gh-(gh-l)*(gh>l)-(gh+l)*(gh<-l)
    p1=gv-(gv-l)*(gv>l)-(gv+l)*(gv<-l)
    temp[0]=p0
    temp[1]=p1
    return temp

In [ ]:
lamb = 100
tau = 0.5
sigma = 1/(8*tau)

I= np.argwhere(Omega==1)
theta = 1

init = np.zeros_like(u0)
xb = init.copy()
y = np.copy(inp_img)
x = init.copy()

def Iteration_CP(x = x ,xb = xb, y = y, theta = theta,lamb = lamb, sigma = sigma):

    pt = y + sigma * Gradient(xb)

    y = pt / (np.maximum(1,npl.norm(pt)))
    
    x_old = np.copy(x)
    
    ut = x_old - tau * Div(y)
    
    x = ut*Omega + (1 - Omega) * (ut + tau * lamb * inp_img)/(1 + tau * lamb)
    
    xb = x + theta*(x - x_old)
    f = npl.norm(xb - u0)
    return x,xb,y,f  

In [ ]:
RES1 = []
RES2 = []

F = []
tstart = time.time()
for i in range(1000):
    x,xb,y,f = Iteration_CP(x,xb,y)
    RES1 += [xb]
    RES2 += [x]

    F += [f]
    if i%50 == 0:
        print("Step : ",i)
print("\n\nTime : ",time.time() - tstart)

In [ ]:
fig, axs = plt.subplots(2, 2,figsize=(20,15))
axs[0, 0].imshow(u0, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
axs[0, 0].set_title('Initial Image')
axs[0, 1].imshow(inp_img, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
axs[0, 1].set_title('Inpainting Image')
axs[1, 0].imshow(xb, cmap=plt.get_cmap('gray'), vmin=0, vmax=1)
axs[1, 0].set_title('Solution')
tmp = RES1[-1]-u0
axs[1, 1].imshow(tmp, cmap=plt.get_cmap('gray'), vmin=np.min(tmp), vmax=np.max(tmp))
axs[1, 1].set_title('Difference')

for ax in axs.flat:
    ax.label_outer()
plt.show()

In [ ]:
%matplotlib auto
plt.figure(0)
plt.plot(F)
plt.show()